In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from denoise_model import denoising_net

In [2]:
def NormalizeData(data):
    return (data - tf.math.reduce_min(data)) / (tf.math.reduce_max(data) - tf.math.reduce_min(data))

In [3]:
def image_selection(enhancement_images):
    images = []
    for index, i in enumerate(enhancement_images):
        r,g,b = tf.split(i, 3, axis=2)
        r_g = int(np.abs(np.mean(r*0.299-g*0.587)))
        r_b = int(np.abs(np.mean(r*0.299-b*0.114)))
        g_b = int(np.abs(np.mean(g*0.587-b*0.114)))
        
        img = tf.image.rgb_to_hsv(i)
        h,s,v = tf.split(img, 3, axis=2)
        mean_hue = tf.math.reduce_mean(h*255)
        mean_v = tf.math.reduce_mean(v)
        
        if 22 < mean_hue and mean_hue <= 100:
            if 30 <= mean_v and mean_v <= 145:
                if  5<=r_g and  r_g <= 36: 
                    if 5 <= r_b and r_b<=48:
                        if 5 <= g_b and  g_b <= 75:
                            images.append(i)
                            print(index)
    if np.shape(images)[0] == 0:
        return enhancement_images[0:5]
    return images 

In [6]:
model_name = ['Small_Denoised_sigmoid_LOL_Lambda2_1']
model_name = model_name[0]
mode = ['Large','Small']
mode = mode[1]
add_noise = False
print(model_name)

Small_Denoised_sigmoid_LOL_Lambda2_1


In [7]:
i = 'epoch1901_ssim0.869741_psnr33.173935'
model = denoising_net(input_shape=(None,None,3), mode=mode, model_name=model_name, train=False, add_noise=add_noise)
# model.enhancement_net.load_weights(enhancement_path)
denoising_path = './model/{0}/weights/{1}/'.format(model_name, i)
model.denoising_net.load_weights(denoising_path)

In [8]:
# img_dir = './Dataset/denoise/train/'
img_dir = './result/CSP_DCE_1_10_5_1600/epoch10/'
output_dir = './result/CSP_DCE+Small_N2N_1_10_5_1600/'
folder = os.path.exists(output_dir)
if not folder:
    os.makedirs(output_dir)

for file in os.listdir(img_dir):
    img_path = img_dir + file
    img = cv2.imread(img_path)
    # cv2.imwrite(output_dir+file, img)
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
    h,w,c = np.shape(img)
    img = tf.reshape(img, (1,h,w,c))
    pred_img = tf.cast(img, dtype=tf.float32)

    denoise, noise = model.predict(img)

    denoise = tf.clip_by_value(denoise, clip_value_min=0, clip_value_max=1)
    # noise = tf.cast(noise,dtype=tf.float32)
    # print(tf.dtypes(noise))

    # # print('img_noise_ssim: ', tf.image.ssim(pred_img,noise,max_val=1))
    # print('img_denoise_ssim: ', tf.image.ssim(pred_img,denoise,max_val=1))
    # print()
    # # print('img_noise_psnr: ', tf.image.psnr(pred_img,noise,max_val=1))
    # print('img_denoise_psnr: ', tf.image.psnr(pred_img,denoise,max_val=1))
    # print()
        
    denoise = np.reshape(denoise, (h,w,c))
    denoise = cv2.cvtColor(denoise, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_dir + file, denoise*255)
    
    # noise = np.reshape(noise, (h,w,c))
    # noise = cv2.cvtColor(noise, cv2.COLOR_RGB2BGR)
    # cv2.imwrite(output_dir + file[:-4] + 'noise.jpg', noise*255)